# Домашнее задание
## Почувствуй мощь трансформеров в бою
В качестве данных выберете возьмите датасет RuCola для русского языка https://github.com/RussianNLP/RuCoLA (в качестве train возьмите in_domain_train.csv, а в качестве теста in_domain_dev.csv)
Разбейте in_domain_train на train и val
Зафайнтьюньте и протестируйте RuBert или RuRoBerta на данной задаче (можно взять любую предобученную модель руберт с сайта huggingface. Например, ruBert-base/large https://huggingface.co/sberbank-ai/ruBert-base / https://huggingface.co/sberbank-ai/ruBert-large или rubert-base-cased https://huggingface.co/DeepPavlov/rubert-base-cased, ruRoberta-large https://huggingface.co/sberbank-ai/ruRoberta-large, xlm-roberta-base https://huggingface.co/xlm-roberta-base)
Возьмите RuGPT3 base или large и решите данное задание с помощью методов few-/zero-shot
а) переберите несколько вариантов затравок
б) протестируйте различное число few-shot примеров (0, 1, 2, 4)
Обучите и протестируйте модель RuT5 на данной задаче (пример finetun’а можете найти здесь https://github.com/RussianNLP/RuCoLA/blob/main/baselines/finetune_t5.py)
Сравните полученные результаты


Критерии оценки:
Файнтьюн БЕРТА - 3 балла
Реализация few-/zero-shot c GPT3 - 4 балла
Файнтьюн RuT5 - 3 балла

In [30]:
!pip install wget evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
!pip install auto_gptq

In [3]:
!git clone https://github.com/RussianNLP/RuCoLA.git

Cloning into 'RuCoLA'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 73 (delta 30), reused 52 (delta 22), pack-reused 0
Receiving objects: 100% (73/73), 944.21 KiB | 3.08 MiB/s, done.
Resolving deltas: 100% (30/30), done.


## Import Dependencies

In [4]:
import numpy as np
import pandas as pd

import torch
from torch.optim import AdamW
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

import transformers
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM
)

from auto_gptq import AutoGPTQForCausalLM

from sklearn.metrics import matthews_corrcoef

import random

from tqdm import tqdm

import matplotlib.pyplot as plt

In [5]:
# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4



## RuBERT Fine-Tuning

In [20]:
train_path = "RuCoLA/data/in_domain_train.csv"
test_path = "RuCoLA/data/in_domain_dev.csv"

train_df = pd.read_csv(train_path, index_col="id")
test_df = pd.read_csv(test_path, index_col="id")

print('Number of training sentences: {:,}\n'.format(train_df.shape[0]))
print('Number of test sentences: {:,}\n'.format(test_df.shape[0]))

train_df.head()

Number of training sentences: 7,869

Number of test sentences: 983



,sentence,acceptable,error_type,detailed_source
id,,,,
0,"Вдруг решетка беззвучно поехала в сторону, и н...",1,0,Paducheva2004
1,Этим летом не никуда ездили.,0,Syntax,Rusgram
2,Только Иван выразил какую бы то ни было готовн...,1,0,Paducheva2013
3,"Теперь ты видишь собственными глазами, как тут...",1,0,Paducheva2010
4,На поверку вся теория оказалась полной чепухой.,1,0,Paducheva2010


In [7]:
model_tag = 'DeepPavlov/rubert-base-cased'
model = BertForSequenceClassification.from_pretrained(
    model_tag,
    num_labels=2,
    output_attentions = False,
    output_hidden_states = False,
    )
model.to(device);

optimizer = AdamW(
    model.parameters(),
    lr=2e-5
)

tokenizer = BertTokenizer.from_pretrained(model_tag, do_lower_case=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [8]:
def create_dataloaders_from_df(tokenizer, train_df, test_df, x_key="sentence", y_key="acceptable", valid_size=0.1, batch_size=32, new_sent_len=64, is_t5=False):
  main_sentences = train_df[x_key]
  main_labels = train_df[y_key]

  main_input_ids = []
  main_att_masks = []

  for sent in main_sentences:
    encoded_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens = True,
        max_length=new_sent_len,
        padding="max_length",
        return_attention_mask = True,
        return_tensors = 'pt',
    )

    main_input_ids.append(encoded_dict['input_ids'])
    main_att_masks.append(encoded_dict['attention_mask'])

  main_input_ids = torch.cat(main_input_ids, dim=0)
  main_att_masks = torch.cat(main_att_masks, dim=0)
  main_labels = torch.tensor(main_labels)

  main_dataset = TensorDataset(main_input_ids, main_att_masks, main_labels)

  valid_len = int(len(main_dataset) * valid_size)
  train_len = len(main_dataset) - valid_len

  train_dataset, valid_dataset = random_split(main_dataset, [train_len, valid_len])

  print(f"Размер тренировочной выборки - {len(train_dataset)}")
  print(f"Размер валидационной выборки - {len(valid_dataset)}")

  train_dataloader = DataLoader(
      train_dataset,
      sampler=RandomSampler(train_dataset),
      batch_size=batch_size,
      drop_last=True
  )

  valid_dataloader = DataLoader(
      valid_dataset,
      sampler=SequentialSampler(valid_dataset),
      batch_size=batch_size
  )

  test_input_ids = []
  test_att_masks = []

  test_sentences = test_df[x_key]
  test_labels = test_df[y_key]

  for sent in test_sentences:
    encoded_dict = tokenizer.encode_plus(
        sent,
        add_special_tokens=True,
        max_length=new_sent_len,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )

    test_input_ids.append(encoded_dict['input_ids'])
    test_att_masks.append(encoded_dict['attention_mask'])

  test_input_ids = torch.cat(test_input_ids, dim=0)
  test_att_masks = torch.cat(test_att_masks, dim=0)
  test_labels = torch.tensor(test_labels)

  test_dataset = TensorDataset(test_input_ids, test_att_masks, test_labels)
  test_dataloader = DataLoader(
      test_dataset,
      sampler=SequentialSampler(test_dataset),
      batch_size=batch_size
  )


  return train_dataloader, valid_dataloader, test_dataloader

def flat_acc(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  return np.sum(preds_flat == labels_flat) / len(labels_flat)

In [9]:
train_dataloader, val_dataloader, prediction_dataloader = create_dataloaders_from_df(tokenizer, train_df, test_df)

Размер тренировочной выборки - 7083
Размер валидационной выборки - 786


### Обучение

In [10]:
# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

train_loss_list = []
valid_loss_list = []

train_acc_list = []
valid_acc_list = []

epochs = 3

for epoch in range(epochs):
  total_train_loss = 0
  total_train_acc = 0

  model.train()
  pbar = tqdm(enumerate(train_dataloader), total=len(train_dataloader))

  for step, sample in pbar:
    model.zero_grad()

    input_ids, attention_masks, labels = sample

    input_ids = input_ids.to(device)
    attention_masks = attention_masks.to(device)
    labels = labels.to(device)

    res = model(
        input_ids=input_ids,
        token_type_ids=None,
        attention_mask=attention_masks,
        labels=labels
    )

    loss = res['loss']
    logits = res['logits']

    curr_loss = loss.item()
    total_train_loss += curr_loss

    curr_acc = flat_acc(logits.cpu().detach().numpy(), labels.cpu().numpy())
    total_train_acc += curr_acc

    pbar.set_description(f"Train Loss: {curr_loss:.4f}; Train Accuracy: {curr_acc:.4f}")

    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()

  avg_train_loss = total_train_loss / len(train_dataloader)
  avg_train_acc = total_train_acc / len(train_dataloader)

  train_loss_list.append(avg_train_loss)
  train_acc_list.append(avg_train_acc)

  print(f"Avg Train Loss: {avg_train_loss:.4f}\nAvg Train Acc: {avg_train_acc:.4f}\n")

  pbar = tqdm(enumerate(val_dataloader), total=len(val_dataloader))

  total_valid_loss = 0
  total_valid_acc = 0

  model.eval()

  with torch.no_grad():

    for step, sample in pbar:
      input_ids, attention_masks, labels = sample

      input_ids = input_ids.to(device)
      attention_masks = attention_masks.to(device)
      labels = labels.to(device)

      res = model(
        input_ids=input_ids,
        token_type_ids=None,
        attention_mask=attention_masks,
        labels=labels
      )

      loss = res['loss']
      logits = res['logits']

      curr_loss = loss.item()
      total_valid_loss += curr_loss

      curr_acc = flat_acc(logits.cpu().detach().numpy(), labels.cpu().numpy())
      total_valid_acc += curr_acc

      pbar.set_description(f"Valid Loss: {curr_loss:.4f}; Valid Accuracy: {curr_acc:.4f}")

  avg_valid_loss = total_valid_loss / len(val_dataloader)
  avg_valid_acc = total_valid_acc / len(val_dataloader)

  valid_loss_list.append(avg_valid_loss)
  valid_acc_list.append(avg_valid_acc)

  print(f"Avg Valid Loss: {avg_valid_loss:.4f}\nAvg Valid Acc: {avg_valid_acc:.4f}\n")

Train Loss: 0.5386; Train Accuracy: 0.8125:  48%|████▊     | 105/221 [00:36<00:40,  2.84it/s]


KeyboardInterrupt: 

### Train and Valid Loss and Accuracy

In [ ]:
def plot_loss_and_accuracy(epochs, train_loss_list, valid_loss_list, train_acc_list, valid_acc_list):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.title("Loss over Epochs")
    plt.plot(range(epochs), train_loss_list, marker="o", label="Train")
    plt.plot(range(epochs), valid_loss_list, marker="o", label="Valid")
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.title("Accuracy over Epochs")
    plt.plot(range(epochs), train_acc_list, marker="o", label="Train")
    plt.plot(range(epochs), valid_acc_list, marker="o", label="Valid")
    plt.legend()
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.grid(True)

    plt.tight_layout()
    plt.show()

plot_loss_and_accuracy(epochs, train_loss_list, valid_loss_list, train_acc_list, valid_acc_list)

### Evaluation on Test data

In [ ]:
# Put model in evaluation mode
model.eval()

# Tracking variables
predictions , true_labels = [], []

# Predict
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

  # Telling the model not to compute or store gradients, saving memory and
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None,
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

In [ ]:
def calc_mcc_from_logits(true_labels, predictions):
  matthews_set = []

  # For each input batch...
  for i in range(len(true_labels)):

    # The predictions for this batch are a 2-column ndarray (one column for "0"
    # and one column for "1"). Pick the label with the highest value and turn this
    # in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()

    # Calculate and store the coef for this batch.
    matthews = matthews_corrcoef(true_labels[i], pred_labels_i)
    matthews_set.append(matthews)

  flat_predictions = np.concatenate(predictions, axis=0)

  # For each sample, pick the label (0 or 1) with the higher score.
  flat_predictions = np.argmax(flat_predictions, axis=1).flatten()

  # Combine the correct labels for each batch into a single list.
  flat_true_labels = np.concatenate(true_labels, axis=0)

  # Calculate the MCC
  mcc = matthews_corrcoef(flat_true_labels, flat_predictions)

  print('Total MCC: %.3f' % mcc)

In [ ]:
calc_mcc_from_logits(true_labels, predictions)

## Few-shot zero-shot RuGPT3

In [ ]:
def calc_loss(phrase: str,
                        tokenizer,
                        model):

    phrase = tokenizer.encode(phrase.lower()) #
    # Если длина фразы 1 токен, то дальше ошибка вылезет :(
    if len(phrase) == 1:
         phrase.append(tokenizer.eos_token_id)
    phrase = torch.tensor(phrase, device=device)
    phrase = phrase.unsqueeze(0)  # .repeat(num_samples, 1)
    with torch.no_grad():
        loss = model(phrase, labels=phrase)

    return loss[0].item()

def get_loss_num(text):
    loss = calc_loss(phrase=text, model=model, tokenizer=tokenizer)
    return loss

def create_prompt_string(sent, label, left_text="В следующем предложении есть ошибка? "):
    right_text = "нет" if label == 1 else "да"
    return f"{left_text}{sent} Ответ: {right_text}."

In [ ]:
def gpt_eval(model, tokenizer, sentences, labels, shots=[]):
    if shots:
        few_shots = [create_prompt_string(sample[0], sample[1]) for sample in shots]
        prompt_prefix = "\n".join(few_shots) + "\n"
    else:
        prompt_prefix = ""

    model.eval()
    global predictions
    global true_labels
    predictions = []
    true_labels = []

    for sent, label in zip(sentences, labels):
        false_sent_loss = calc_loss(prompt_prefix + create_prompt_string(sent, 0), tokenizer, model)
        true_sent_loss = calc_loss(prompt_prefix + create_prompt_string(sent, 1), tokenizer, model)

        loss_tuple = (false_sent_loss, true_sent_loss)
        predictions.append(np.argmin(loss_tuple))
        true_labels.append(label)

    mcc = matthews_corrcoef(true_labels, predictions)
    print(f"Matthews correlation coefficient on shots={len(shots)}: {mcc}")

In [ ]:
model_name = 'IlyaGusev/rugpt3medium_sum_gazeta'

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)
# model.cuda()

# shots = [
#     ("Собака лает на прохожих.", 1),
#     ("Человек идет в и магазине.", 0),
#     ("Они гулять по парке.", 0),
#     # ("Книга лежит на столе.", 1),
#     # ("Книга любить читать.", 0)
# ]

# gpt_eval(
#     model=model,
#     tokenizer=tokenizer,
#     sentences=test_df.sentence,
#     labels=test_df.acceptable,
#     shots=shots
# )

# gpt_eval(
#     model=model,
#     tokenizer=tokenizer,
#     sentences=test_df.sentence,
#     labels=test_df.acceptable,
# )

### Сводка по Few-shot Zero-shot GPT3

В данном случае у промптов была следующая структура:

> Предложение далее корректное? {Предложение} Ответ: {да/нет}.

Пробовал другие структуры , по типу
> Следующее предложение корректное? {Предложение} Ответ: {да/нет}.

> Следующее предложение содержит ошибки? {Предложение} Ответ: {нет/да}.

Особо никаких результатов не принесло. Из моделей ai-forever/rugpt3{small/medium/large}_based_on_gpt2 лучший результат показала large. Показатели следующие:
Структура промпта:
> В следующем предложении есть ошибка? {Предложение} Ответ: {нет/да}.

Shot'ы:
```
shots = [
    ("Собака лает на прохожих.", 1),
    ("Человек идет в и магазине.", 0),
    ("Они гулять по парку.", 0),
    ("Книга лежит на столе.", 1),
    ("Книга любить читать.", 0)
]
```

Показатель MCC на тестовой выборке: 0.06881427127451528

Это не особо хороший результат(где-то 55/100, т.к. коэф. корреляции Мэтьюса лежит в диапазоне от -1 до 1, где 0 - случайное подкидывание монетки), но хотя бы выше, чем если просто случайно выбирать 0 или 1.

Остальные попытки с разными моделями/промтами/кол-вом промптов не дали лучшего результата.

Пробовал IlyaGusev/rugpt3medium_sum_gazeta -  это просто medium модель сбера, которую дообучили на задаче суммаризации газет.
```
shots = [
    ("Собака лает на прохожих.", 1),
    ("Человек идет в и магазине.", 0),
    ("Они гулять по парке.", 0),
]
```
Показатель MCC на тестовой выборке: 0.08441909656772244

Уже получше, но всё равно не очень хороший результат.


Однако инференс ооооочень долгий - порядка 30 минут. Для сравнения, на small-medium-large моделях от Сбера инференс длился 1-2 минуты.


По поводу экспериментов с количеством few-shot примеров и их разнообразием: если много примеров - плохо (видимо уже зашумляется промпт и модель плохо реагирует на целевой), а разнообразие примеров тяжеловато подобрать, потому что русский язык весьма сложный, нужно охватить синтаксические, семантические и морфологические ошибки(и в каждой из этих групп весьма много примеров).

Также модель весьма сильно реагирует на то, с каким количеством ошибок подаётся ошибочное предложение для примера(оптимальное - две, т.е если больше или меньше, уже отрицательно сказывается на результате).


В целом - надо просто иметь хорошие модельки(что-то типо GPT 3.5 от Сбера) - и немного поэкспериментировать с промптами, и в таком случае, несложные задачи, подобные этой, можно вполне решать.

А с моделями типо gpt-2 я считаю такое их применение нецелесообразным (видимо, слабоватые модели для такого серьёзного подхода)






## RuT5 Fine-Tuning

In [11]:
model_name = "ai-forever/ruT5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [15]:
train_df.head()

,sentence,acceptable
id,,
0,"Вдруг решетка беззвучно поехала в сторону, и н...",1
1,Этим летом не никуда ездили.,0
2,Только Иван выразил какую бы то ни было готовн...,1
3,"Теперь ты видишь собственными глазами, как тут...",1
4,На поверку вся теория оказалась полной чепухой.,1


In [25]:
from datasets import Dataset, DatasetDict

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

train_df["text"] = train_df["sentence"]
test_df["text"] = test_df["sentence"]

train_df["label"] = train_df["acceptable"].astype(str)
test_df["label"] = test_df["acceptable"].astype(str)

train_df = train_df.drop(columns=["acceptable", "error_type", "detailed_source", "id", "sentence"])
test_df = test_df.drop(columns=["acceptable", "error_type", "detailed_source", "id", "sentence"])

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset
})

dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 7869
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 983
    })
})

In [26]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["text"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["label"], truncation=True), batched=True, remove_columns=['text', 'label'])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/8852 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Max source length: 48


Map:   0%|          | 0/8852 [00:00<?, ? examples/s]

Max target length: 2


In [28]:
def preprocess_function(sample, padding="max_length"):
    # add prefix to the input for t5
    inputs = [item for item in sample["text"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["label"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=['text', 'label'])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/7869 [00:00<?, ? examples/s]

Map:   0%|          | 0/983 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [31]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("f1")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, average='macro')
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [33]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = 0
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [34]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Hugging Face repository id
# repository_id = f"{model_id.split('/')[1]}-imdb-text-classification"
repository_id="./repo"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=3e-4,

    num_train_epochs=4,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="epoch",
    # logging_steps=1000,
    evaluation_strategy="no",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=False,
    # metric_for_best_model="overall_f1",
    # push to hub parameters

    # report_to="tensorboard",
    # push_to_hub=True,
    # hub_strategy="every_save",
    # hub_model_id=repository_id,
    # hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
trainer.train()

Step,Training Loss
984,0.118600
1968,0.069600


TrainOutput(global_step=1968, training_loss=0.09412948096670755, metrics={'train_runtime': 554.5047, 'train_samples_per_second': 28.382, 'train_steps_per_second': 3.549, 'total_flos': 898479307653120.0, 'train_loss': 0.09412948096670755, 'epoch': 2.0})

In [36]:
trainer.evaluate()

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'eval_loss': 0.06598323583602905,
 'eval_f1': 56.6468,
 'eval_gen_len': 2.0,
 'eval_runtime': 12.0965,
 'eval_samples_per_second': 81.263,
 'eval_steps_per_second': 10.168,
 'epoch': 2.0}